In [1]:
'''
Plano de ação do MNIST
1. Prepare os dados e pré-processe. Crie datasets de treinamento, validação e teste.
2. Defina o modelo e escolha as funções de ativação.
3. Defina os otimizadores avançados adequados e a função de perda.
4. Faça-o aprender.
5. Teste a precisão do modelo usando o dataset de teste.
'''

'\nPlano de ação do MNIST\n1. Prepare os dados e pré-processe. Crie datasets de treinamento, validação e teste.\n2. Defina o modelo e escolha as funções de ativação.\n3. Defina os otimizadores avançados adequados e a função de perda.\n4. Faça-o aprender.\n5. Teste a precisão do modelo usando o dataset de teste.\n'

In [2]:
# Libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
# Datset
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [4]:
# Datset Preprocessing
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [5]:
# Model Implementation
# Model Hyperparameters: inputLayer = 784; hiddenLayers: 2 X 100; outputLayer: 10
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(output_size, activation="softmax")
                            ])

In [6]:
# Optimizer
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [7]:
# Training
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 16s - loss: 0.3305 - accuracy: 0.9057 - val_loss: 0.1693 - val_accuracy: 0.9502 - 16s/epoch - 29ms/step
Epoch 2/5
540/540 - 7s - loss: 0.1339 - accuracy: 0.9607 - val_loss: 0.1061 - val_accuracy: 0.9700 - 7s/epoch - 13ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0941 - accuracy: 0.9729 - val_loss: 0.1016 - val_accuracy: 0.9702 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0723 - accuracy: 0.9771 - val_loss: 0.0775 - val_accuracy: 0.9778 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0561 - accuracy: 0.9827 - val_loss: 0.0701 - val_accuracy: 0.9777 - 4s/epoch - 7ms/step


In [8]:
# Model Test (model deployment)
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test loss: {test_loss}. Test accuracy: {test_accuracy}")

1/1 [==============================] - 1s 809ms/step - loss: 0.0836 - accuracy: 0.9753
Test loss: 0.08355484902858734. Test accuracy: 0.9753000140190125
